In [1]:
import pandas as pd
import sys
sys.path.append('../src/')
%matplotlib inline
import numpy as np
from utils.visual_functions import *
from net.metrics import compute_regress_metrics, compute_metrics, example_f1_score
from data.load_data import ukdale_appliance_data as appliance_data
from sklearn import metrics
import math
import numpy as np
from sklearn.metrics import f1_score, mean_absolute_error, mean_squared_error
from collections import OrderedDict

def get_mae(target, prediction):
    return mean_absolute_error(target, prediction)

def get_eac(target, prediction):
    num = np.sum(np.abs(prediction-target))
    den = (np.sum(target))
    eac = 1 - (num/den)/2
    return np.where(eac<0, 0, eac)

def get_relative_error(target, prediction):
    return np.mean(np.nan_to_num(np.abs(target - prediction) / np.maximum(target, prediction)))


def get_nde(target, prediction):
    return np.sum((target - prediction) ** 2) / np.sum((target ** 2))

def compute_regress_metrics(target, prediction):
    eac = get_eac(target, prediction)
    mae = get_mae(target, prediction)
    nade = get_nde(target, prediction)
    
   
    metrics = OrderedDict([('EAC', eac),
                        ('MAE', mae),
                        ('NDE', nade)])
    
    metrics_dict = {}
    metrics_dict['EAC'] = metrics["EAC"]
    metrics_dict['MAE'] = metrics["MAE"]
    metrics_dict['NDE'] = metrics["NDE"]
    
    return metrics_dict

appliances=['kettle', 'fridge', 'dishwasher', 'washingmachine', 'microwave'] 

In [2]:
#result_unet = np.load("../results/ukdale_UNETNiLM_quantilesresults.npy", allow_pickle=True).item()
result_cnn = np.load("../results/ukdale_CNN1D_quantiles_multi-applianceresults.npy", allow_pickle=True).item()

In [3]:
pred_power=result_cnn['pred_power']
pred_state=result_cnn['pred_state']
power=result_cnn['power']
state=result_cnn['state']

In [5]:
def smape_score(target, prediction):
    denom = np.abs(prediction) + np.abs(target) 
    score =(2 * np.abs(prediction - target).mean(0))/(denom.mean(0) + 1e-8)
    return score  

In [31]:
y_t = power 
y_p =  pred_power[:,2,:]
def get_CEP(y_t, y_p):
    c = np.sum((y_t == y_p).astype(int)*y_p, 0)
    ce = np.sum((y_t > y_p).astype(int)*y_t, 0)
    co = np.sum((y_t < y_p).astype(int)*y_p, 0)
    gt = np.sum(y_t, 0)
    CEP = (c + ce + co)[gt!=0]/(gt[gt!=0])
    CEP[gt==0]=1.0
    return CEP
gt = np.sum(y_t, 0)
O = np.sum( (y_p - y_t), 0)
U = np.sum( (y_t - y_p), 0)

OE = O[gt!=0]/(gt[gt!=0])
OE

array([-0.02520563,  0.7804243 , -0.3360108 ,  0.02802461, -0.07546163],
      dtype=float32)

In [7]:
smape_score(power, pred_power[:,2,:])

array([0.52876055, 0.6509373 , 0.45955464, 0.22291598, 0.33214587],
      dtype=float32)

In [ ]:
112.193428  24.682268  103.999756       34.784824  35.651764

In [27]:
pred_power=result_unet['pred_power']
pred_state=result_unet['pred_state']
power=result_unet['power']
state=result_unet['state']
all_reg_results_mean = []
all_reg_results_std = []
regress = {}
for idx, app in enumerate(list(appliance_data.keys())):
    regress_results = []
    for  q in range(5):
        result = compute_regress_metrics(power[:,idx]*state[:,idx], pred_power[:,q,idx]*pred_state[:,idx])
        result = pd.DataFrame.from_dict(result, orient="index")
        regress_results.append(result)
    regress_results = pd.concat(regress_results, axis=1)
    #regress_results.columns = list(appliance_data.keys())
    #regress[app]=regress_results
    print("APP", app)
    mean = regress_results.mean(1).round(3)
    std  = regress_results.std(1).round(3)
    reg=pd.concat([mean, std], axis=1)
    reg.columns =["mean", 'std']
    all_reg_results_mean.append( mean)
    all_reg_results_std.append(std)
    print(reg)
    print("")
#ebf1 = {"ExbF1":example_f1_score(state, pred_state, axis=0, per_sample=True).tolist()}
#ebf1 =pd.DataFrame.from_dict(ebf1, orient="index")
#ebf1.columns = list(appliance_data.keys())
print("")
#print(ebf1.round(3))


#print(pd.DataFrame.from_dict(compute_metrics(state, pred_state), orient="index"))

APP kettle
       mean    std
EAC   0.677  0.017
MAE  16.003  0.824
NDE   0.429  0.039

APP fridge
       mean    std
EAC   0.937  0.000
MAE  15.124  0.014
NDE   0.072  0.000

APP dishwasher
      mean    std
EAC  0.914  0.000
MAE  6.764  0.012
NDE  0.080  0.000

APP washingmachine
       mean    std
EAC   0.909  0.000
MAE  11.506  0.006
NDE   0.062  0.000

APP microwave
      mean    std
EAC  0.753  0.003
MAE  6.475  0.072
NDE  0.334  0.005




In [17]:
pd.concat(all_reg_results_mean, axis=1).mean(1).round(3)

EAC     0.927
MAE    36.340
NDE     0.100
dtype: float64

## CNN results

In [20]:
result_cnn = np.load("../results/ukdale_CNN1D_quantiles_multi-applianceresults.npy", allow_pickle=True).item()

In [34]:
result_cnn['avg_results']

,ukdale
EAC,0.923241
NDE,6.081102
MAE,30.002014
exbF1,0.643800
maF1,0.643800
miF1,0.778200
HA,0.954628


In [32]:
pred_power=result_cnn['pred_power']
pred_state=result_cnn['pred_state']
power=result_cnn['power']
state=result_cnn['state']
all_reg_results_mean = []
all_reg_results_std = []
regress = {}
for idx, app in enumerate(appliances):
    regress_results = []
    for  q in range(3):
        result = compute_regress_metrics(power[:,idx], pred_power[:,q,idx])
        result = pd.DataFrame.from_dict(result, orient="index")
        regress_results.append(result)
    regress_results = pd.concat(regress_results, axis=1)
    #regress_results.columns = list(appliance_data.keys())
    #regress[app]=regress_results
    print("APP", app)
    mean = regress_results.mean(1).round(3)
    std  = regress_results.std(1).round(3)
    reg=pd.concat([mean, std], axis=1)
    reg.columns =["mean", 'std']
    all_reg_results_mean.append( mean)
    all_reg_results_std.append(std)
    print(reg)
    print("")
ebf1 = {"ExbF1":example_f1_score(state, pred_state, axis=0, per_sample=True).tolist()}
ebf1 =pd.DataFrame.from_dict(ebf1, orient="index")
ebf1.columns = appliances
print("")
print(ebf1.round(3))
#print(pd.DataFrame.from_dict(compute_metrics(state, pred_state), orient="index"))

APP kettle
        mean     std
EAC    0.721   0.083
MAE  125.097  37.214
NDE    0.484   0.190

APP fridge
      mean    std
EAC  0.939  0.020
MAE  6.355  2.099
NDE  0.263  0.093

APP dishwasher
       mean    std
EAC   0.922  0.008
MAE  33.477  3.285
NDE   0.516  0.089

APP washingmachine
       mean    std
EAC   0.955  0.013
MAE  22.009  6.356
NDE   0.340  0.166

APP microwave
       mean    std
EAC   0.931  0.027
MAE  18.561  7.291
NDE   0.524  0.187


       kettle  fridge  dishwasher  washingmachine  microwave
ExbF1   0.767   0.773       0.585           0.928      0.167


In [20]:
pd.concat(all_reg_results_mean, axis=1).mean(1).round(3)

EAC     0.947
MAE    19.340
NDE     0.127
dtype: float64

In [21]:
pd.concat(all_reg_results_std, axis=1).mean(1).round(3)

EAC    0.002
MAE    1.260
NDE    0.001
dtype: float64

## Visual

In [188]:
y_p_cnn = result_cnn['pred_power'][41160:41172+5000]
y_t_cnn = result_cnn['power'][41160:41172+5000]
z_t_cnn = result_cnn['state'][41160:41172+5000]
z_p_cnn = result_cnn['pred_state'][41160:41172+5000]

In [189]:
appliance_labels={'kettle':"KT", 'fridge':"FRZ", 'dishwasher':"DW", 'washingmachine':"WM", 'microwave':"MW"}
fig = plt.figure(figsize=(6, 4))
for i, app in enumerate(list(appliance_data.keys())):
    #plt.plot(y_t_cnn[:,i], label=appliance_labels[app], color=colors[i])
    plt.plot(y_p_cnn[:,2,i]*z_p_cnn[:,i], label=appliance_labels[app], color=colors[i], linestyle="--")
plt.ylabel("Power $W$")
plt.legend()
savefig("cnn_prediction", format=".pdf")

In [187]:
appliance_labels={'kettle':"KT", 'fridge':"FRZ", 'dishwasher':"DW", 'washingmachine':"WM", 'microwave':"MW"}
fig = plt.figure(figsize=(6, 4))
for i, app in enumerate(list(appliance_data.keys())):
    plt.plot(y_t_cnn[:,i]*z_t_cnn[:,i], label=appliance_labels[app], color=colors[i])
    #plt.plot(y_p_cnn[:,2,i]t label=appliance_labels[app], color=colors[i], linestyle="--")
plt.ylabel("Power $W$")
plt.legend()
savefig("ground_truth", format=".pdf")